# Initialize

In [ ]:
!pip show pandas

In [2]:
import requests
import mysql.connector
from mysql.connector import Error
import os

In [3]:
#calling data functions from file = 'mod02_data_functions.py', based on notebook md02
from mod02_data_functions import get_mysql_cursor, list_mysql_databases_and_tables, create_covertype_table, delete_covertype_table,\
fetch_all_covertype_records, get_data_from_api, insert_unique_covertype_data, print_random_10_with_headers


In [4]:
import mod03A_preprocessing

In [5]:
import mod04_train_model

In [ ]:
get_mysql_cursor()

In [ ]:
list_mysql_databases_and_tables()

## Remove previous tables

In [ ]:
#delete_covertype_table('raw_data')
delete_covertype_table('clean_data')

# Create Tables - Raw and Clean data

In [ ]:
raw_data_table = 'raw_data'

In [ ]:
#create_covertype_table(raw_data_table)

In [ ]:
#create_covertype_table(clean_data_table)

# Get data from API, save it into raw data table

In [ ]:
#Get batch data from API
group_number = 2 #there are 10 groups in total
batch_data = get_data_from_api(group_number)

In [ ]:
#Save batch_data into raw data table
insert_unique_covertype_data(batch_data, raw_data_table)

In [ ]:
#validate there is data in raw_table
fetch_all_covertype_records(raw_data_table)

# Pre-processing

1. Data Cleaning
Handle missing values: Although the dataset typically has no missing values, it's good practice to check

2. Remove duplicates: Ensure no repeated rows.

3. Feature Engineering
Normalize distances: Scale distance-related features to reduce skew.

Horizontal_Distance_To_Hydrology
Vertical_Distance_To_Hydrology
Horizontal_Distance_To_Roadways
Horizontal_Distance_To_Fire_Points
Hillshade interaction: Create a feature like average hil

df['Hillshade_Avg'] = df[['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']].mean(axis=1)
4. Encoding Categorical Variables
Wilderness_Area and Soil_Type are categorical.
Use One-Hot Encoding if using tree-based models.
Use Label Encoding if using linear models or neural networ

5. Feature Scaling
Apply StandardScaler or MinMaxScaler to numerical features:
Elevation, Aspect, Slope, et
**************************************
6. Train-Test Split
Split the dataset before modeling

from sklearn.model_selection import train_test_split
X = df.drop("Cover_Type", axis=1)
y = df["Cover_Type"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
:c.ks.
lshade:.

In [ ]:
#confirm packages
!pip show scikit-learn

In [ ]:
mod03A_preprocessing.main()

In [ ]:
records = fetch_all_covertype_records('clean_data')

In [ ]:
print_random_10_with_headers('clean_data')

In [ ]:
#headers
"""raw_id  Elevation    Aspect     Slope  Hillshade_9am  Hillshade_Noon  Hillshade_3pm  Hillshade_Avg  Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  Horizontal_Distance_To_Fire_Points  Wilderness_Area_Cache  Wilderness_Area_Commanche  Wilderness_Area_Neota  Wilderness_Area_Rawah  Soil_Type_C2702  Soil_Type_C2703  Soil_Type_C2704  Soil_Type_C2705  Soil_Type_C2706  Soil_Type_C2717  Soil_Type_C3501  Soil_Type_C4201  Soil_Type_C4703  Soil_Type_C4704  Soil_Type_C4744  Soil_Type_C4758  Soil_Type_C6101  Soil_Type_C6102  Soil_Type_C6731  Soil_Type_C7101  Soil_Type_C7102  Soil_Type_C7103  Soil_Type_C7201  Soil_Type_C7202  Soil_Type_C7700  Soil_Type_C7702  Soil_Type_C7709  Soil_Type_C7710  Soil_Type_C7745  Soil_Type_C7746  Soil_Type_C7755  Soil_Type_C7756  Soil_Type_C7757  Soil_Type_C7790  Soil_Type_C8703  Soil_Type_C8708  Soil_Type_C8771  Soil_Type_C8772  Soil_Type_C8776  Cover_Type
"""

# Training Classification models

In [ ]:
model = train_cover_type_model()

# MLFlow - register model and experiments

***Rule of thumb - Conections between containers***

a. Inside container-to-container communication (e.g., another service in the same Docker network): use container name and container port:

"http://mlflow:5000"

b. From host or anything outside Docker: use host port (5001 in your mapping):

"http://localhost:5001"

In [ ]:
#!pip show mlflow
%pip install --upgrade typing_extensions


***IMPORTANT: install boto3 in jupyter***

Even if your MLflow container has boto3, the client doing mlflow.sklearn.log_model() must also have it, because MLflow uploads artifacts directly from the client to S3.

In [ ]:
#boto3 version in mlflow container: boto3: 1.40.2
#installing the same version here in jupyter
#%pip install boto3==1.40.2


In [ ]:
import mlflow
import mlflow.sklearn
import os
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import boto3

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")


In [ ]:
import boto3
print("boto3 version:", boto3.__version__)

## Experiment 1

In [ ]:

# loads the diabetes dataset
db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# run description (just metadata)
desc = "the simplest possible example"


# Connect to MLflow tracking server defined in docker-compose file for mlflow service

mlflow.set_tracking_uri("http://mlflow:5000") # docker-compose, mlflow service, ports: 5001:5000--> host:container
mlflow.set_experiment("mlflow_tracking_examples_class")

with mlflow.start_run(run_name="logged_artifacts") as run:
    params = {"n_estimators":50, "max_depth":6, "max_features":3}

    rf = RandomForestRegressor(**params)
    rf.fit(X_train, y_train)

    mlflow.log_params(params)
    mlflow.sklearn.log_model(
      sk_model=rf,
      artifact_path="random_forest_regressor"
    )

## Experiment 2

In [ ]:
#initial batch_data from group 2
mod04_train_model.main()

## Experiment 3 - adding new batch_data

In [ ]:
#Get batch data from API
group_number = 2 #there are 10 groups in total
batch_data = get_data_from_api(group_number)

In [ ]:
#Save batch_data into raw data table
insert_unique_covertype_data(batch_data, raw_data_table)

In [ ]:
#validate there is data in raw_table
fetch_all_covertype_records(raw_data_table)

In [1]:
#PRE-PROCESSING - applied to new batch_data
#this initial version is breaking the schema due to one/hot/encoder
#mod03A_preprocessing.main()

NameError: name 'mod03A_preprocessing' is not defined

In [6]:
#importing v2 of preprocessing module
#this was refactored to skip one hot encoding, which was breaking the schema
import mod03_preprocessing_v2

In [7]:
#clearing old data
delete_covertype_table('clean_data')

MySQL cursor created successfully
Table 'clean_data' deleted if it existed.


In [8]:
#preprocessing with new version

mod03_preprocessing_v2.main()

MySQL cursor created successfully


/home/jovyan/work/mod03_preprocessing_v2.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


MySQL cursor created successfully
Inserted 2324 new processed rows into 'clean_data'.


In [9]:
#Running experiment with new data
mod04_train_model.main()

MySQL cursor created successfully
Accuracy: 0.7526881720430108

Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.79      0.75       174
           1       0.76      0.77      0.76       213
           2       0.87      0.85      0.86        40
           3       1.00      0.50      0.67         2
           4       0.00      0.00      0.00        11
           5       1.00      0.33      0.50         9
           6       0.79      0.69      0.73        16

    accuracy                           0.75       465
   macro avg       0.73      0.56      0.61       465
weighted avg       0.74      0.75      0.74       465



/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2025/08/08 19:30:07 WARNING mlflow.models.model: `artifact_path`

MLflow Run ID: 38fbfdfe13c9498c96746d9af9f9aa06
🏃 View run RandomForest_CoverType at: http://mlflow:5000/#/experiments/2/runs/38fbfdfe13c9498c96746d9af9f9aa06
🧪 View experiment at: http://mlflow:5000/#/experiments/2
